In [1]:
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt
from cnn_env import CnnSnekEnv
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [2]:
env = CnnSnekEnv()
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order="last")

c:\Users\robby\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [3]:
class TrainingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose: int = 0):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if (self.save_path is not None):
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, "model_{}".format(self.n_calls))
            self.model.save(model_path)
        return True

In [4]:
CHECKPOINT_DIR = "./train/"
LOG_DIR = "./logs/"
callback = TrainingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [5]:
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.00001, n_steps=1024)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [6]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_11
----------------------------
| time/              |     |
|    fps             | 59  |
|    iterations      | 1   |
|    time_elapsed    | 8   |
|    total_timesteps | 512 |
----------------------------


KeyboardInterrupt: 

## Test the model

In [ ]:
# Load model
model = PPO.load("./train/file")

In [ ]:
state = env.reset()
while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()